# Lab 6: การดึงข้อมูลข่าวจาก Backend API (Client-side และ Server-side Fetching)

---

## วัตถุประสงค์

เพื่อเรียนรู้การดึงข้อมูลจาก API ภายนอกในแอป Next.js โดยเปรียบเทียบระหว่างการดึงข้อมูลฝั่ง Client (useEffect) และฝั่ง Server (Server Component)

---

## โครงสร้างโปรเจกต์ (เฉพาะส่วนที่เกี่ยวข้อง)

```
src/
├── app/
│   └── (content)/
│       └── news/
│           └── page.js         # ใช้ทั้งแบบ Client และ Server Component
└── backend/
    ├── app.js                  # API Backend แบบ Express
    └── package.json
```

---

## ส่วนที่ 1: ดึงข้อมูลด้วย Client Component (useEffect)

### ขั้นตอนที่ 1. เริ่มต้นเซิร์ฟเวอร์ Backend

1.1 เปิดเทอร์มินัลใหม่ แล้วเข้าสู่โฟลเดอร์ `backend`:

```bash
cd backend
```

1.2 ติดตั้ง dependency ของ backend:

```bash
npm install
```

1.3 เริ่มต้นเซิร์ฟเวอร์ backend:

```bash
npm start
```

1.4 เปิดเทอร์มินัลอีกหน้าต่างหนึ่ง ไปยังโฟลเดอร์หลักของโปรเจกต์:

```bash
cd ..
npm run dev
```

### ขั้นตอนที่ 2. ใช้ไฟล์ `page.js` ที่เตรียมไว้ (Client-side Fetch)

```jsx
"use client";

import { useEffect, useState } from 'react';
import NewsList from '@/components/news-list';

export default function NewsPage() {
  const [isLoading, setIsLoading] = useState(false);
  const [error, setError] = useState();
  const [news, setNews] = useState();

  useEffect(() => {
    async function fetchNews() {
      setIsLoading(true);
      const response = await fetch('http://localhost:8080/news');
      if (!response.ok) {
        setError('ดึงข้อมูลข่าวไม่สำเร็จ');
        setIsLoading(false);
        return;
      }
      const news = await response.json();
      setIsLoading(false);
      setNews(news);
    }
    fetchNews();
  }, []);

  if (isLoading) return <p>Loading...</p>;
  if (error) return <p>{error}</p>;

  return (
    <>
      <h1>หน้ารายการข่าว</h1>
      {news && <NewsList news={news} />}
    </>
  );
}
```

### คำอธิบาย

* `useEffect`: เรียกใช้หลัง component mount (โหลดข้อมูลฝั่ง client)
* `useState`: จัดการสถานะโหลด, error และข้อมูลข่าว
* เหมาะสำหรับข้อมูลที่ไม่สำคัญต่อ SEO หรือเปลี่ยนแปลงตาม interaction

---

## ส่วนที่ 2: ดึงข้อมูลด้วย Server Component (Server-side Fetch)

### ขั้นตอนที่ 1. ปรับ `page.js` ให้เป็น Server Component

```jsx
import NewsList from '@/components/news-list';

export default async function NewsPage() {
  const response = await fetch('http://localhost:8080/news');
  if (!response.ok) {
    throw new Error('ดึงข้อมูลข่าวไม่สำเร็จ');
  }
  const news = await response.json();
  return (
    <>
      <h1>หน้ารายการข่าว</h1>
      <NewsList news={news} />
    </>
  );
}
```

### คำอธิบาย

* ลบ `"use client"` และ import ที่ไม่จำเป็น (`useEffect`, `useState`)
* ใช้ `fetch()` ได้โดยตรงใน Server Component
* ทำงานฝั่ง server → มีข้อมูลพร้อมตั้งแต่โหลดหน้าแรก
* เหมาะสำหรับ SEO และลด flicker ของ UI

---

## เปรียบเทียบผลลัพธ์ระหว่างสองวิธี

| ด้าน             | Client-side (useEffect) | Server-side (Server Component) |
| ---------------- | ----------------------- | ------------------------------ |
| เหมาะสำหรับ      | ข้อมูลแบบ real-time     | เนื้อหาที่ต้องการ SEO          |
| Render HTML      | ไม่มีข้อมูลข่าว         | มีข้อมูลข่าวใน source HTML     |
| ใช้ใน React ปกติ | ได้                     | ไม่ได้ (เฉพาะ Next.js)         |
| การกระพริบ       | อาจเกิดขึ้น             | ไม่มี                          |

---

## ส่วนที่ 3: ดึงข้อมูลจากฐานข้อมูลโดยตรง (Direct Database Access จาก Server Component)

ในกรณีที่แอปของเราควบคุมทั้งฝั่ง frontend และ backend เราอาจไม่จำเป็นต้องมี backend server แยกต่างหากเลย เราสามารถย้ายไฟล์ `data.db` มาไว้ในโฟลเดอร์โปรเจกต์ Next.js และเข้าถึง SQLite โดยตรงจาก Server Component ได้

### ขั้นตอนที่ 1: ติดตั้งไลบรารี `better-sqlite3`

```bash
npm install better-sqlite3
```

### ขั้นตอนที่ 2: ย้ายไฟล์ `data.db` มาที่โฟลเดอร์โปรเจกต์ Next.js (ระดับราก)

### ขั้นตอนที่ 3: สร้างไฟล์ `src/lib/news.js`

```js
import SQL from 'better-sqlite3';

const db = new SQL('data.db');

export function getAllNews() {
  const news = db.prepare('SELECT * FROM news').all();
  return news;
}
```

### ขั้นตอนที่ 4: แก้ไข `page.js` ให้ใช้ `getAllNews()` แทนการ fetch

```jsx
import { getAllNews } from '@/lib/news';
import NewsList from '@/components/news-list';

export default function NewsPage() {
  const news = getAllNews();

  return (
    <>
      <h1>หน้ารายการข่าว</h1>
      <NewsList news={news} />
    </>
  );
}
```

### คำอธิบาย

* เราไม่ต้องส่ง HTTP request ไปยัง backend อีกต่อไป
* Server Component สามารถเข้าถึงฐานข้อมูลได้โดยตรง เพราะทำงานบน server
* ช่วยลด latency และความซับซ้อนของระบบ
* ไม่สามารถใช้ใน Client Component ได้ เพราะ client ไม่สามารถเข้าถึงฐานข้อมูลโดยตรง

---

## เปรียบเทียบแนวทางทั้งสาม

| ด้าน        | Client-side (useEffect)   | Server-side (fetch API) | Server-side (Database Access) |
| ----------- | ------------------------- | ----------------------- | ----------------------------- |
| เหมาะสำหรับ | ข้อมูลเปลี่ยนบ่อย/ภายหลัง | เนื้อหาที่ต้องการ SEO   | ระบบควบคุม backend เอง        |
| ความเร็ว    | ช้า (โหลดหลังแสดงผล)      | เร็ว (โหลดพร้อม render) | เร็วที่สุด (ไม่มี HTTP)       |
| ใช้ HTTP    | ใช่                       | ใช่                     | ไม่ใช้                        |
| โค้ดง่าย    | ปานกลาง                   | ง่าย                    | ง่าย + ต้องใช้ SQLite         |

---

## ส่วนที่ 4: การจำลองความล่าช้า และการแสดงสถานะโหลดด้วย `loading.js`

บางครั้งเราอาจต้องการจำลองสถานการณ์ที่การโหลดข้อมูลช้าลง เช่น ฐานข้อมูลตอบกลับช้า เพื่อทดสอบการแสดงผล fallback ของหน้าเพจ

### ขั้นตอนที่ 1. ปรับฟังก์ชัน `getAllNews()` ให้เป็น async และเพิ่ม delay

ใน `src/lib/news.js`:

```js
import SQL from 'better-sqlite3';

const db = new SQL('data.db');

export async function getAllNews() {
  await new Promise((resolve) => setTimeout(resolve, 2000)); // ดีเลย์ 2 วินาที
  const news = db.prepare('SELECT * FROM news').all();
  return news;
}
```

### ขั้นตอนที่ 2. ปรับ `page.js` ให้รองรับ await

```jsx
import { getAllNews } from '@/lib/news';
import NewsList from '@/components/news-list';

export default async function NewsPage() {
  const news = await getAllNews();

  return (
    <>
      <h1>หน้ารายการข่าว</h1>
      <NewsList news={news} />
    </>
  );
}
```

### ขั้นตอนที่ 3. สร้างไฟล์ `loading.js` เพื่อแสดงผลระหว่างโหลดข้อมูล

สร้างในโฟลเดอร์เดียวกับ `page.js` (`src/app/(content)/news/loading.js`):

```jsx
export default function LoadingNews() {
  return <p>กำลังโหลดข่าว โปรดรอสักครู่...</p>;
}
```

เมื่อผู้ใช้เข้าหน้า `/news` ระบบจะแสดงข้อความใน `loading.js` ทันที แล้วรอจนกว่าข้อมูลจากฐานข้อมูลจะโหลดเสร็จ

### ผลลัพธ์ที่ได้:

* หน้าโหลดแสดงผลอย่างราบรื่น
* ป้องกันการคลิกแล้วเงียบเฉยไม่มี feedback
* ช่วยให้ประสบการณ์ผู้ใช้ดีขึ้น

---

## ส่วนที่ 5: ปรับปรุงส่วนอื่น ๆ ของแอปให้ใช้ฐานข้อมูลแทน dummy-news

### 1. ใช้ฟังก์ชันใหม่จากไฟล์ `news.js`

อัปเดตจากไฟล์ที่แนบ `news.js` แล้ว (ดูรายละเอียดในโค้ดด้านล่าง) เราสามารถใช้ฟังก์ชันใหม่ เช่น `getNewsItem`, `getLatestNews`, `getNewsForYearAndMonth`, `getAvailableNewsYears`, `getAvailableNewsMonths` ได้ทั้งหมด ซึ่งเชื่อมต่อฐานข้อมูล SQLite โดยตรงและมีการ delay จำลองไว้เพื่อให้ทดสอบ fallback ได้

### 2. การเปลี่ยน `page.js` ใน `/news/[slug]/page.js`

```jsx
import { getNewsItem } from '@/lib/news';

export default async function NewsDetailPage({ params }) {
  const newsItem = await getNewsItem(params.slug);

  return (
    <article>
      <h1>{newsItem.title}</h1>
      <time>{newsItem.date}</time>
      <p>{newsItem.content}</p>
    </article>
  );
}
```

### 3. เพิ่ม fallback เฉพาะสำหรับ slug

ใน `news/[slug]/loading.js`:

```jsx
export default function LoadingNewsItem() {
  return <p>กำลังโหลดเนื้อหาข่าว...</p>;
}
```

### 4. Intercepted Routes และ useRouter

ในกรณีที่เรามี **Intercepted Route** เช่น `/news/image/[slug]` ซึ่งแสดง Modal แทนการเปลี่ยนหน้าเต็ม ผู้ใช้จะกดที่ภาพข่าวแล้วมี Popup ขึ้นแสดงรูปหรือเนื้อหาข่าว ซึ่งเบื้องหลังใช้ client-side routing ผ่าน `useRouter()`

#### 4.1 ปัญหา: ไม่สามารถใช้ `useRouter()` ใน Server Component ได้

`useRouter()` ใช้ได้เฉพาะใน Client Component เท่านั้น หากเราเขียนใน `page.js` ที่เป็น Server Component จะเกิดข้อผิดพลาด หรือทำให้ไม่สามารถใช้ `async/await` ดึงข้อมูลได้

#### 4.2 แนวทางแก้ไข: แยก Client Component ออกมา

1. สร้างไฟล์ใหม่ `src/components/modal-backdrop.js`

```jsx
'use client';
import { useRouter } from 'next/navigation';

export default function ModalBackdrop() {
  const router = useRouter();
  return (
    <div className="backdrop" onClick={() => router.back()} />
  );
}
```

2. แก้ไข `page.js` ใน `/news/image/[slug]/page.js` ให้เป็น Server Component อีกครั้ง

```jsx
import { getNewsItem } from '@/lib/news';
import ModalBackdrop from '@/components/modal-backdrop';

export default async function NewsImagePage({ params }) {
  const newsItem = await getNewsItem(params.slug);

  return (
    <>
      <ModalBackdrop />
      <img src={newsItem.image} alt={newsItem.title} />
    </>
  );
}
```

### 4.3 ทำให้ Next.js รู้ว่า route นี้คือ intercepted modal

เปลี่ยนชื่อไฟล์ `page.js` ภายในโฟลเดอร์ `@modal` → เป็น `default.js`

```
src/app/(content)/news/@modal/[slug]/default.js
```

> หมายเหตุ: การใช้ชื่อ `default.js` เป็นข้อบังคับของ Next.js สำหรับ parallel/intercepted routes

#### 4.4 Fallback loading เฉพาะจุด

สามารถเพิ่มไฟล์ `loading.js` หรือใช้ `<Suspense>` เพื่อแสดงข้อความ "กำลังโหลดรูปภาพ..." ระหว่างโหลดข้อมูลของ intercepted modal ได้ด้วยเช่นกัน

### 5. การใช้ Suspense สำหรับ fallback

เราจะใช้ `React.Suspense` เพื่อแสดง fallback เฉพาะจุดที่กำลังโหลดข้อมูล เช่น เฉพาะ header หรือเฉพาะข่าว โดยมีขั้นตอนดังนี้:

#### 5.1 กรณีใช้งาน: หน้า Filter ข่าว (archive/filter/\[year]/\[month]/page.js)

* เรามีข้อมูล 2 ส่วนที่โหลดช้า: 1) รายการข่าว, 2) header ที่มีปี/เดือน

#### 5.2 สร้าง component แยก: `FilteredNews.js`

```jsx
import { getNewsForYear, getNewsForYearAndMonth } from '@/lib/news';
import NewsList from '@/components/news-list';

export default async function FilteredNews({ year, month }) {
  let news;
  if (year && !month) {
    news = await getNewsForYear(year);
  } else if (year && month) {
    news = await getNewsForYearAndMonth(year, month);
  } else {
    news = [];
  }

  if (!news || news.length === 0) {
    return <p>ไม่มีข่าวสำหรับช่วงเวลานี้</p>;
  }

  return <NewsList news={news} />;
}
```

#### 5.3 สร้าง component แยก: `FilterHeader.js`

```jsx
import { getAvailableNewsYears, getAvailableNewsMonths } from '@/lib/news';
import Link from 'next/link';

export default async function FilterHeader({ year, month }) {
  const availableYears = await getAvailableNewsYears();
  const availableMonths = year ? getAvailableNewsMonths(year) : [];

  if (year && !availableYears.includes(year)) {
    throw new Error('ปีไม่ถูกต้อง');
  }
  if (month && !availableMonths.includes(month)) {
    throw new Error('เดือนไม่ถูกต้อง');
  }

  return (
    <header>
      <h2>ปี: {year} เดือน: {month || '-'}</h2>
      {/* ลิงก์สำหรับปี/เดือน */}
    </header>
  );
}
```

#### 5.4 ปรับ page.js ให้ใช้ Suspense แยก fallback แต่ละส่วน

```jsx
import { Suspense } from 'react';
import FilteredNews from './FilteredNews';
import FilterHeader from './FilterHeader';

export default function FilterPage({ params }) {
  const { year, month } = params;

  return (
    <>
      <Suspense fallback={<p>กำลังโหลดหัวข้อ...</p>}>
        <FilterHeader year={year} month={month} />
      </Suspense>

      <Suspense fallback={<p>กำลังโหลดข่าว...</p>}>
        <FilteredNews year={year} month={month} />
      </Suspense>
    </>
  );
}
```

#### 5.5 ประโยชน์ที่ได้

* เพิ่มความละเอียดในการควบคุม fallback
* แต่ละ component สามารถโหลดและแสดงผลได้ทันทีเมื่อข้อมูลพร้อม
* ผู้ใช้รับรู้ได้ว่าระบบกำลังทำงาน แม้ยังโหลดข้อมูลไม่เสร็จ

การใช้ `Suspense` เป็นทางเลือกที่ยืดหยุ่นและแนะนำสำหรับการแสดงสถานะโหลดที่แม่นยำใน Next.js เช่น ส่วน header กับ ส่วนรายการข่าว

---